In [1]:
import xgboost
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('Data/xgboost_classification.csv', sep=';')
data

,age,response
0,23,0
1,45,0
2,44,0
3,20,0
4,49,0
5,56,1
6,55,1
7,43,1
8,23,0
9,67,0


In [3]:
y = data.pop('response').values
X = data

In [4]:
dtrain = xgboost.DMatrix(data = X, label = y)
params = {'max_depth': 1, 'eta': 1.0, 'objective': 'binary:logistic',
          'lambda': 0, 'base_score': 0.5}
model = xgboost.train(params=params, num_boost_round=1, dtrain=dtrain)
pr = model.predict(dtrain)
print(pr, '\n\n', model.get_dump()[0])

[14:27:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[0.28224894 0.28224894 0.28224894 0.28224894 0.28224894 0.59868765
 0.59868765 0.28224894 0.28224894 0.59868765 0.28224894 0.28224894
 0.28224894 0.28224894 0.28224894 0.28224894 0.28224894 0.59868765
 0.59868765 0.28224894] 

 0:[age<55] yes=1,no=2,missing=1
	1:leaf=-0.933333337
	2:leaf=0.400000006



In [5]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

base_score = 0.5
leaf_score_for_less_than_55 = -0.933333337
leaf_score_for_55_and_more = 0.400000006

prediction_for_less_than_55 = sigmoid(
    np.log(base_score / (1 - base_score)) + leaf_score_for_less_than_55)
prediction_for_55_and_more = sigmoid(
    np.log(base_score / (1 - base_score)) + leaf_score_for_55_and_more)
print('прогноз для age <55', prediction_for_less_than_55)
print('прогноз для age >=55', prediction_for_55_and_more)

прогноз для age <55 0.28224894229945025
прогноз для age >=55 0.5986876615540165


In [6]:
model = xgboost.train(params=params, num_boost_round=2, dtrain=dtrain)
pr = model.predict(dtrain)
print(pr)
for line in model.get_dump(): 
    print(line)

[14:27:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[14:27:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[0.41287592 0.1725412  0.1725412  0.41287592 0.1725412  0.44166934
 0.44166934 0.41287592 0.41287592 0.44166934 0.1725412  0.41287592
 0.41287592 0.41287592 0.1725412  0.41287592 0.41287592 0.44166934
 0.44166934 0.41287592]
0:[age<55] yes=1,no=2,missing=1
	1:leaf=-0.933333337
	2:leaf=0.400000006

0:[age<44] yes=1,no=2,missing=1
	1:leaf=0.581244171
	2:leaf=-0.634389997



In [7]:
base_score = 0.5
leaf_score_for_less_than_55 = -0.933333337
leaf_score_for_55_and_more = 0.400000006

leaf_score_for_less_than_44 = 0.581244171
leaf_score_for_44_and_more = -0.634389997

prediction_for_obs_0 = sigmoid(
    np.log(base_score / (1 - base_score)) + (leaf_score_for_less_than_55 + leaf_score_for_less_than_44))
prediction_for_obs_1 = sigmoid(
    np.log(base_score / (1 - base_score)) + (leaf_score_for_less_than_55 + leaf_score_for_44_and_more))
print('прогноз для наблюдения с индексом 0', prediction_for_obs_0)
print('прогноз для наблюдения с индексом 1', prediction_for_obs_1)

прогноз для наблюдения с индексом 0 0.4128758956142221
прогноз для наблюдения с индексом 1 0.17254119071402557


In [8]:
data = pd.read_csv('Data/xgboost_regression.csv', sep=';')
data

,age,days_of_delinq
0,23,6
1,45,8
2,44,10
3,20,12
4,49,15
5,56,10
6,55,14
7,43,20
8,23,11
9,67,6


In [9]:
y = data.pop('days_of_delinq').values
X = data

In [10]:
dtrain = xgboost.DMatrix(data = X, label = y)
params = {'max_depth': 1, 'eta': 1.0, 'objective': 'reg:linear', 'lambda': 0, 'base_score': 0.5}
model = xgboost.train(params=params, num_boost_round=1, dtrain=dtrain)
pr = model.predict(dtrain)
print(pr, '\n\n', model.get_dump()[0])

[14:27:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[12.375 12.375 12.375 12.375 12.375  6.    12.375 12.375 12.375  6.
 12.375 12.375 12.375 12.375 12.375 12.375 12.375  6.     6.    12.375] 

 0:[age<56] yes=1,no=2,missing=1
	1:leaf=11.875
	2:leaf=5.5



In [11]:
base_score = 0.5
leaf_score_for_less_than_56 = 11.875
leaf_score_for_56_and_more = 5.5
prediction_for_less_than_56 = base_score + leaf_score_for_less_than_56
prediction_for_56_and_more = base_score + leaf_score_for_56_and_more
print('прогноз для age <56', prediction_for_less_than_56)
print('прогноз для age >=56', prediction_for_56_and_more)

прогноз для age <56 12.375
прогноз для age >=56 6.0


In [12]:
params = {'max_depth': 1, 'eta': 0.5, 'objective': 'reg:linear', 'lambda': 0, 'base_score': 0.5}
model = xgboost.train(params=params, num_boost_round=1, dtrain=dtrain)
pr = model.predict(dtrain)
print(pr, '\n\n', model.get_dump()[0])

[14:27:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[6.4375 6.4375 6.4375 6.4375 6.4375 3.25   6.4375 6.4375 6.4375 3.25
 6.4375 6.4375 6.4375 6.4375 6.4375 6.4375 6.4375 3.25   3.25   6.4375] 

 0:[age<56] yes=1,no=2,missing=1
	1:leaf=5.9375
	2:leaf=2.75



In [13]:
base_score = 0.5
leaf_score_for_less_than_56 = 5.9375
leaf_score_for_56_and_more = 2.75
prediction_for_less_than_56 = base_score + leaf_score_for_less_than_56
prediction_for_56_and_more = base_score + leaf_score_for_56_and_more
print('прогноз для age <56', prediction_for_less_than_56)
print('прогноз для age >=56', prediction_for_56_and_more)

прогноз для age <56 6.4375
прогноз для age >=56 3.25


In [14]:
params = {'max_depth': 1, 'eta': 0.5, 'objective': 'reg:linear', 'lambda': 0, 'base_score': 0.3}
model = xgboost.train(params=params, num_boost_round=1, dtrain=dtrain)
pr = model.predict(dtrain)
print(pr, '\n\n', model.get_dump()[0])

[14:27:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[6.3375    6.3375    6.3375    6.3375    6.3375    3.1499999 6.3375
 6.3375    6.3375    3.1499999 6.3375    6.3375    6.3375    6.3375
 6.3375    6.3375    6.3375    3.1499999 3.1499999 6.3375   ] 

 0:[age<56] yes=1,no=2,missing=1
	1:leaf=6.0374999
	2:leaf=2.8499999



In [15]:
base_score = 0.3
leaf_score_for_less_than_56 = 6.0374999
leaf_score_for_56_and_more = 2.8499999
prediction_for_less_than_56 = base_score + leaf_score_for_less_than_56
prediction_for_56_and_more = base_score + leaf_score_for_56_and_more
print('прогноз для age <56', prediction_for_less_than_56)
print('прогноз для age >=56', prediction_for_56_and_more)

прогноз для age <56 6.3374999
прогноз для age >=56 3.1499998999999996


In [16]:
params = {'max_depth': 1, 'eta': 1.0, 'objective': 'reg:linear', 'lambda': 0, 'base_score': 0.5}
model = xgboost.train(params=params, num_boost_round=2, dtrain=dtrain)
pr = model.predict(dtrain)
print(pr)
for line in model.get_dump(): 
    print(line)

[14:27:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[14:27:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[10.285714 13.5      13.5      10.285714 13.5       7.125    13.5
 13.5      10.285714  7.125    13.5      13.5      10.285714 10.285714
 13.5      13.5      10.285714  7.125     7.125    10.285714]
0:[age<56] yes=1,no=2,missing=1
	1:leaf=11.875
	2:leaf=5.5

0:[age<35] yes=1,no=2,missing=1
	1:leaf=-2.08928561
	2:leaf=1.125



In [17]:
base_score = 0.5
leaf_score_for_less_than_56_tree1 = 11.875
leaf_score_for_56_and_more_tree1 = 5.5
leaf_score_for_less_than_35_tree2 = -2.08928561
leaf_score_for_35_and_more_tree2 = 1.125
prediction_for_obs0 = base_score + leaf_score_for_less_than_56_tree1 + leaf_score_for_less_than_35_tree2
prediction_for_obs1 = base_score + leaf_score_for_less_than_56_tree1 + leaf_score_for_35_and_more_tree2
print('прогноз для наблюдения с индексом 0', prediction_for_obs0)
print('прогноз для наблюдения с индексом 1', prediction_for_obs1)

прогноз для наблюдения с индексом 0 10.285714389999999
прогноз для наблюдения с индексом 1 13.5


In [18]:
params = {'max_depth': 1, 'eta': 0.9, 'objective': 'reg:linear', 'lambda': 0, 'base_score': 0.5}
model = xgboost.train(params=params, num_boost_round=2, dtrain=dtrain)
pr = model.predict(dtrain)
print(pr)
for line in model.get_dump(): 
    print(line)

[14:27:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[14:27:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[10.375893  13.092212  13.092212  10.375893  13.092212   7.3547115
 13.092212  13.092212  10.375893   7.3547115 13.092212  13.092212
 10.375893  10.375893  13.092212  13.092212  10.375893   7.3547115
  7.3547115 10.375893 ]
0:[age<56] yes=1,no=2,missing=1
	1:leaf=10.6875
	2:leaf=4.94999981

0:[age<35] yes=1,no=2,missing=1
	1:leaf=-0.811607122
	2:leaf=1.90471148



In [19]:
# поскольку eta = 0.9, значения листьев умножаются на 0.9,
# например, 11,875 x 0,9 = 10,6875

base_score = 0.5
leaf_score_for_less_than_56_tree1 = 10.6875
leaf_score_for_56_and_more_tree1 = 4.94999981
leaf_score_for_less_than_35_tree2 = -0.811607122
leaf_score_for_35_and_more_tree2 = 1.90471148
prediction_for_obs0 = base_score + leaf_score_for_less_than_56_tree1 + leaf_score_for_less_than_35_tree2
prediction_for_obs1 = base_score + leaf_score_for_less_than_56_tree1 + leaf_score_for_35_and_more_tree2
print('прогноз для наблюдения с индексом 0', prediction_for_obs0)
print('прогноз для наблюдения с индексом 1', prediction_for_obs1)

прогноз для наблюдения с индексом 0 10.375892878
прогноз для наблюдения с индексом 1 13.09221148


In [20]:
params = {'max_depth': 2, 'eta': 0.5, 'objective': 'reg:linear', 'lambda': 0, 'base_score': 0.5}
model = xgboost.train(params=params, num_boost_round=1, dtrain=dtrain)
pr = model.predict(dtrain)
print(pr, '\n\n', model.get_dump()[0])

[14:27:04] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[5.392857 7.25     7.25     5.392857 7.25     3.25     7.25     7.25
 5.392857 3.25     7.25     7.25     5.392857 5.392857 7.25     7.25
 5.392857 3.25     3.25     5.392857] 

 0:[age<56] yes=1,no=2,missing=1
	1:[age<35] yes=3,no=4,missing=3
		3:leaf=4.89285707
		4:leaf=6.75
	2:leaf=2.75



In [21]:
base_score = 0.5
leaf_score_for_less_than_35 = 4.89285707
leaf_score_for_35_55 = 6.75
leaf_score_for_56_and_more = 2.75
prediction_for_less_than_35 = base_score + leaf_score_for_less_than_35
prediction_for_35_55 = base_score + leaf_score_for_35_55
prediction_for_56_and_more = base_score + leaf_score_for_56_and_more
print('прогноз для age < 35', prediction_for_less_than_35)
print('прогноз для age 35-55', prediction_for_35_55)
print('прогноз для age >=56', prediction_for_56_and_more)

прогноз для age < 35 5.39285707
прогноз для age 35-55 7.25
прогноз для age >=56 3.25
